In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MAPE
from tensorflow.keras import callbacks
from google.cloud import storage
import joblib

In [36]:
def subsample_sequence(df, length):
    '''
    function that return a random slice of features and targets
    len(X) = lenght and len(y) = 3
    '''
    last_possible = df.shape[0] - length - 3
    random_start = np.random.randint(0, last_possible)
    X = df[random_start: random_start+length].values
    y = df.iloc[random_start+length:random_start+length+3][['price_usd']]
    return X, y

In [37]:
def get_X_y(df, length_of_observations):
    '''
    function that returns a list of random slices of features and targets
    len(X[0]) = lenght and len(y[0]) = 3
    '''
    X, y = [], []
    for length in length_of_observations:
        xi, yi = subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
    return X, y

In [38]:
def split_tr_te(df, horizon=3, ratio=0.95):
    '''
    function that returns a training and test set
    arguments are:
    the horizon of prediction
    the ratio of the train/test split
    '''
    # the gap to avoid data leakage
    # gap = horizon - 1
    # len_ = int(ratio*df.shape[0])
    # data_train = df[:len_]
    # data_test = df[len_+gap:]
    # return data_train, data_test
    
    len_ = int(ratio*df.shape[0])
    data_train = df[:len_]
    data_test = df[len_+1:]
    return data_train, data_test
    
    # len_ = 1696
    # data_train = df[:len_]
    # data_test = df[len_:]
    # return data_train, data_test

In [39]:
def extract_xy_tr_te(train,
                     test,
                     train_splits = 3000,
                     train_time_min = 30,
                     train_time_max = 180):
    '''
    function returns a serie of train and test data
    train splits is the number of selections of our dataset
    train_time_min is the minimum number of days that are randomly choosen by the get_X_y function
    train_time_max is the maximum number of days that are randomly choosen by the get_X_y function
    '''
    length_of_observations = np.random.randint(train_time_min, train_time_max, train_splits)
    X_train, y_train = get_X_y(train, length_of_observations)
    #length_of_observations = np.random.randint(train_time_min, train_time_max, train_splits)
    #X_test, y_test = get_X_y(test, length_of_observations)
    return X_train, y_train #, X_test, y_test


In [40]:
def padding_seq(train):
    '''
    function that return the padded version of the train dataset
    to uniform the size of the model imput
    '''
    return pad_sequences(train, dtype='float32', value=-1)

In [41]:
def baseline_model(X_train_pad, y_train):
    '''
    function that return a trained baseline model and its fitting history
    and save locally the trained model file basemodel.joblib
    '''
    normalizer = Normalization()
    normalizer.adapt(X_train_pad)
    model = Sequential()
    model.add(normalizer)
    model.add(layers.Masking(mask_value=-1))
    model.add(layers.LSTM(40, activation='tanh')) # GS N UNITS
    model.add(layers.Dense(40, activation='relu')) # GS N UNITS
    model.add(layers.Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=RMSprop(learning_rate=0.01), metrics=MAPE)
    es = callbacks.EarlyStopping(patience=6, restore_best_weights=True)
    history = model.fit(X_train_pad,
                np.array(y_train),
                epochs=30,
                batch_size=64,
                validation_split=0.3,
                callbacks=[es],
                verbose=1)
    joblib.dump(model, 'basemodel.joblib')
    return model, history

In [42]:

def plot_history(history, title='', axs=None, exp_name=""):
    '''
    return the loss and metric plots of train and test fit process
    '''
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label = 'train' + exp_name)
    ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 100000)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['mean_absolute_percentage_error'], label='train mape'  + exp_name)
    ax2.plot(history.history['val_mean_absolute_percentage_error'], label='val mape'  + exp_name)
    ax1.set_ylim(0., 100000)
    ax2.set_title('mape')
    ax2.legend()
    return (ax1, ax2)

In [43]:
def pred_3d_price(model, test):
    '''
    return the prediction of three days after the test data
    '''
    return model.predict(test)

---

# Beginning of Grid Search

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MAPE
from tensorflow.keras import callbacks
from google.cloud import storage
import joblib

In [19]:
data = pd.read_csv('../raw_data/data_advanced.csv', index_col = 'datetime') # TO RUN LOCALLY

In [23]:
def train_model(data, nb_sequences=2000, nb_days=21, plot_history=False):

    data.shape
    # data.set_index('datetime', inplace=True)
    # for x in data.columns:
    #     if x == 'Unnamed: 0':
    #         data.drop(columns = x, inplace=True)

    # Beginning of test set
    y_test = np.resize(np.array(data.tail(3)['price_usd']), (3, 1))
    data = data[:-3]
    X_test = data.tail(80)

    ratio=0.95
    len_ = int(ratio*data.shape[0])
    data_train = data[:len_]
    # data_test = data[len_+1:]

    train = data_train
    # test = data_test 
    
    train_splits = 3000
    train_time_min = 30
    train_time_max = 180
    length_of_observations = np.random.randint(train_time_min, train_time_max, train_splits)

    def subsample_sequence(df, length):
        last_possible = df.shape[0] - length - 3
        random_start = np.random.randint(0, last_possible)
        X = df[random_start: random_start+length].values
        y = df.iloc[random_start+length:random_start+length+3][['price_usd']]
        return X, y

    X, y = [], []
    for length in length_of_observations:
        xi, yi = subsample_sequence(data, length)
        X.append(xi)
        y.append(yi)
    

    X_train = X
    y_train = y

    
    X_train_pad = pad_sequences(X_train, dtype='float32', value=-1)


    normalizer = Normalization()
    normalizer.adapt(X_train_pad)

    model = Sequential()
    model.add(normalizer)
    model.add(layers.Masking(mask_value=-1))
    model.add(layers.LSTM(40, activation='tanh')) # GS N UNITS
    model.add(layers.Dense(40, activation='relu')) # GS N UNITS
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', 
                optimizer=RMSprop(learning_rate=0.01), 
                metrics=MAPE)


    es = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(X_train_pad,
                np.array(y_train),
                epochs=30,
                batch_size=64,
                validation_split=0.3,
                callbacks=[es],
                verbose=1)

    if plot_history:
        plt.plot(history.history['mean_absolute_percentage_error'])
        plt.plot(history.history['val_mean_absolute_percentage_error'])
        plt.show()


    y_pred = model.predict(X_test)
    rmse = (((y_test - y_pred)**2).mean())**0.5
    # res = model.evaluate(X_test, y_test, verbose=0)
    return rmse

In [24]:
data

,price_usd,volume_usd,twitter_followers,daily_opening_price_usd,daily_high_price_usd,daily_low_price_usd,daily_closing_price_usd,daily_avg_price_usd,daily_trading_volume_usd,marketcap_usd,...,eth_NFT_buyers,total_NFT_sales,total_NFT_buyers,total_TVL,eth_TVL,000001.SS,^GSPC,^IXIC,^N100,^NDX
datetime,,,,,,,,,,,,,,,,,,,,,
2017-07-17,193.424000,1.669500e+09,3.145176e+03,159.991000,194.753000,158.044000,193.424000,172.514333,1.669500e+09,1.805774e+10,...,5,1628.20,5,0.000000e+00,0.000000e+00,3219.791016,2459.500000,6320.350098,1010.400024,5845.819824
2017-07-18,234.391000,2.709260e+09,4.717765e+03,195.027000,267.986000,182.059000,234.391000,204.764931,2.709260e+09,2.188796e+10,...,11,3998.08,11,0.000000e+00,0.000000e+00,3159.731934,2455.879883,6304.819824,1007.530029,5832.919922
2017-07-19,199.703000,2.328790e+09,6.290353e+03,234.941000,245.651000,191.745000,199.703000,225.754365,2.328790e+09,1.865361e+10,...,5,2808.81,5,0.000000e+00,0.000000e+00,3181.402100,2463.850098,6363.240234,1004.760010,5898.399902
2017-07-20,227.265000,2.225000e+09,7.862941e+03,205.422000,233.904000,203.216000,227.265000,219.248451,2.225000e+09,2.123365e+10,...,4,1721.93,4,0.000000e+00,0.000000e+00,3227.506104,2475.560059,6396.459961,1012.309998,5929.580078
2017-07-21,218.305000,1.006130e+09,9.435529e+03,226.061000,235.118000,213.010000,218.305000,224.663132,1.006130e+09,2.040171e+10,...,6,2523.11,6,0.000000e+00,0.000000e+00,3236.587891,2467.399902,6383.049805,1005.869995,5905.600098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08,1230.209202,1.628545e+10,2.666409e+06,1237.558858,1261.776012,1200.632216,1230.209202,1233.697254,1.628545e+10,1.494304e+11,...,12672,17967380.49,46655,7.807354e+10,4.952661e+10,3380.370117,3888.260010,11503.610352,1169.170044,11977.950195
2022-07-10,1168.401611,1.098456e+10,2.671018e+06,1216.625268,1216.797657,1157.175540,1168.401611,1182.985909,1.098456e+10,1.419541e+11,...,13205,19310139.95,40092,7.804470e+10,4.895357e+10,3360.735107,3884.599976,11514.050293,1162.010010,11993.354980
2022-07-11,1097.236515,1.206418e+10,2.672475e+06,1167.707855,1169.193825,1095.126569,1097.236515,1142.838904,1.206418e+10,1.333228e+11,...,14355,17449920.08,43668,7.637071e+10,4.779272e+10,3341.100098,3880.939941,11524.490234,1154.849976,12008.759766


In [25]:
train_model(data = data, nb_sequences=2000, nb_days=21, plot_history=False)

2022-07-15 23:28:37.293004: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30
33/33 [==============================] - 8s 140ms/step - loss: 2472089.2500 - mean_absolute_percentage_error: 70.6669 - val_loss: 2329602.5000 - val_mean_absolute_percentage_error: 61.6696
Epoch 2/30
33/33 [==============================] - 4s 120ms/step - loss: 1867735.2500 - mean_absolute_percentage_error: 44.2251 - val_loss: 1616584.6250 - val_mean_absolute_percentage_error: 36.7221
Epoch 3/30
33/33 [==============================] - 4s 134ms/step - loss: 1209444.0000 - mean_absolute_percentage_error: 28.8137 - val_loss: 926456.8125 - val_mean_absolute_percentage_error: 24.1266
Epoch 4/30
33/33 [==============================] - 4s 129ms/step - loss: 651208.8750 - mean_absolute_percentage_error: 20.5469 - val_loss: 415814.8750 - val_mean_absolute_percentage_error: 21.5939
Epoch 5/30
33/33 [==============================] - 5s 140ms/step - loss: 268432.6250 - mean_absolute_percentage_error: 14.4514 - val_loss: 131654.2500 - val_mean_absolute_percentage_error: 13.5104
Epoch

551.9666962808463

In [26]:


nb_days_grid = [11, 21, 41, 61, 81, 121, 151]
nb_sequences_grid = [1000, 2000, 4000]
MAPE = []
for nb_days in nb_days_grid:
    for nb_sequences in nb_sequences_grid:
        res = train_model(data = data,nb_sequences=2000, nb_days=nb_days)
        MAPE.append((nb_days, nb_sequences, res))
    
    print(f'For {nb_sequences} sequences and {nb_days} days of observation, the MAPE is of {res:.0f} %')

MAPE

x = [res[0] for res in MAPE] # n_days
y = [res[1] for res in MAPE] # n_seq
c = [res[2] for res in MAPE] # MAPE  
plt.scatter(x, y, c=c, norm=matplotlib.colors.Normalize(vmin=20, vmax=50, clip=False), cmap='coolwarm')

Epoch 1/30
33/33 [==============================] - 8s 164ms/step - loss: 2462288.5000 - val_loss: 2298478.2500
Epoch 2/30
33/33 [==============================] - 4s 133ms/step - loss: 1718887.6250 - val_loss: 1463196.5000
Epoch 3/30
33/33 [==============================] - 4s 132ms/step - loss: 981890.2500 - val_loss: 749334.1875
Epoch 4/30
33/33 [==============================] - 4s 132ms/step - loss: 437806.3750 - val_loss: 341611.1250
Epoch 5/30
33/33 [==============================] - 4s 136ms/step - loss: 175199.2031 - val_loss: 114462.8438
Epoch 6/30
33/33 [==============================] - 4s 135ms/step - loss: 57121.8164 - val_loss: 33481.1875
Epoch 7/30
33/33 [==============================] - 4s 135ms/step - loss: 27926.2246 - val_loss: 20596.3320
Epoch 8/30
33/33 [==============================] - 4s 137ms/step - loss: 22599.9590 - val_loss: 59097.1367
Epoch 9/30
33/33 [==============================] - 4s 136ms/step - loss: 23062.9941 - val_loss: 22803.4531
Epoch 10/30
33

ValueError: in user code:

    File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/engine/training.py", line 864, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/engine/training.py", line 957, 

def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)

# define config
'''
A list of index offsets in the prior observations 
relative to the time to be forecasted that will be used as the prediction. 
For example, 12 will use the observation 12 months ago (-12) relative to the time to be forecasted.
'''
cfg_list = [1, 6, 12, 24, 36]

ratio=0.95
len_ = int(ratio * data.shape[0])

# grid search configs
def grid_search(data, cfg_list, len_):
	# evaluate configs
	scores = [score_model(data, len_, cfg) for cfg in cfg_list]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

In [30]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [45]:
data = pd.read_csv('../raw_data/data_advanced.csv', index_col = 'datetime') # TO RUN LOCALLY

In [32]:
def create_model():
    
    ###########################
    # 1. Define architecture  #
    ###########################    
         # Notice that we don't specify the input shape yet...
         # ... as we don't know the shape post-preprocessing!
         # One consequence is that here, you cannot yet print 
         # the model's summary. It will be known after fitting it
         # to X_train_preprocessed, y_train
        
    normalizer = Normalization()
    normalizer.adapt(X_train_pad)
    model = Sequential()
    model.add(normalizer)
    model.add(layers.Masking(mask_value = -1))
    model.add(layers.LSTM(40, activation='tanh')) # GS N UNITS
    model.add(layers.Dense(40, activation='relu')) # GS N UNITS
    model.add(layers.Dense(1, activation='linear'))

    ###########################    
    # 2. Compile model        #
    ###########################    
    model.compile(loss='mse', 
                    optimizer=RMSprop(learning_rate=0.01), 
                    metrics=MAPE)
    
    return model



In [50]:
disguised_deep_model = KerasRegressor(build_fn = create_model(), 
                                       epochs = 10, 
                                       batch_size = 32, 
                                       verbose = 0)

/var/folders/0k/592h1xbs6sb54xxswdffplgc0000gn/T/ipykernel_44942/1020509451.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  disguised_deep_model = KerasRegressor(build_fn = create_model(),


In [51]:
data_train, data_test = split_tr_te(data)

In [52]:
X_train, y_train = extract_xy_tr_te(train=data_train,
                                    test=data_test)
X_train_pad = padding_seq(X_train)

In [53]:
from sklearn.model_selection import cross_val_score

# $CHALLENGIFY_BEGIN
cross_val_score(disguised_deep_model, 
                X_train_pad, 
                y_train, 
                cv = 3, 
                n_jobs = -1)
# $CHALLENGIFY_END

2022-07-16 00:17:55.707285: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://21a6a90b-dbd9-4212-892f-d0d0889aa929/assets


INFO:tensorflow:Assets written to: ram://21a6a90b-dbd9-4212-892f-d0d0889aa929/assets
2022-07-16 00:18:01.524247: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:01.540904: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:01.697273: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:01.713463: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:01.827540: W tensorflow/core/common_

INFO:tensorflow:Assets written to: ram://f33f7bff-1f6d-4cf7-9a74-b291f7ec78c3/assets


INFO:tensorflow:Assets written to: ram://f33f7bff-1f6d-4cf7-9a74-b291f7ec78c3/assets
2022-07-16 00:18:09.275903: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:09.527866: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:10.717398: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:10.734915: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:10.948653: W tensorflow/core/common_

...

INFO:tensorflow:Assets written to: ram://d68fc200-3f97-442b-8107-57e73f47c1f0/assets


INFO:tensorflow:Assets written to: ram://d68fc200-3f97-442b-8107-57e73f47c1f0/assets
2022-07-16 00:18:36.860154: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:36.891675: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:37.371251: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:37.421772: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:39.089787: W tensorflow/core/common_

INFO:tensorflow:Assets written to: ram://7f4e59b2-0b48-47b9-9ad7-486bdaaf3fc0/assets


INFO:tensorflow:Assets written to: ram://7f4e59b2-0b48-47b9-9ad7-486bdaaf3fc0/assets
2022-07-16 00:18:49.894486: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:49.924460: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:50.004342: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:50.040025: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-07-16 00:18:50.248264: W tensorflow/core/common_

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/wrappers/scikit_learn.py", line 152, in fit
    self.model = self.build_fn(
  File "/Users/jasonhou/.pyenv/versions/3.8.12/envs/Crypto-Analysis/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/Users/jasonhou/.pyenv/versions/3.8.12/envs

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
pipeline = make_pipeline(disguised_deep_model)
pipeline

In [ ]:
pipeline.get_params()

In [ ]:
def create_model_grid(activation = 'relu', 
                      optimizer = 'rmsprop'):
    # create model
    normalizer = Normalization()
    normalizer.adapt(X_train_pad)

    model = Sequential()
    model.add(normalizer)
    model.add(layers.Masking(mask_value=-1))
    model.add(layers.LSTM(40, activation='tanh')) # GS N UNITS
    model.add(layers.Dense(40, activation='relu')) # GS N UNITS
    model.add(layers.Dense(1, activation='linear'))
    
    # Compile model
    model.compile(loss='mse', 
                    optimizer=RMSprop(learning_rate=0.01), 
                    metrics=MAPE)
    return model

In [ ]:
model_grid = KerasRegressor(
                    build_fn = create_model_grid,
                    epochs = 10,
                    batch_size = 32,
                    verbose = 0,
)